## Build datasets

Dataset of multiple text descriptions for each image index. Additional column that has wikipedia columns for random sampling.

In [3]:
import pandas as pd
import geopandas as gpd
import sys
sys.path.append('/scratch/cher/Sat2Habitat/data_prep/geocell') 
# from gbif_utils import _clean_data


In [57]:
grid_key_path = '/scratch/cher/Sat2Habitat/data/grid_key_0.01deg.csv'
habitat_info_path = '/scratch/cher/Sat2Habitat/data/occurrence.txt'
species_wiki_path = '/scratch/cher/Sat2Habitat/data/species_wiki.csv'

In [ ]:
grid_key_df = pd.read_csv(grid_key_path)

species_wiki_df = pd.read_csv(species_wiki_path)
species_wiki_df.rename(columns={col: f"{col}_wiki" if col != 'species' else col for col in species_wiki_df.columns}, inplace=True)

In [131]:
# clean occurrences
habitat_info = pd.read_csv(habitat_info_path, sep="\t", on_bad_lines='skip')
habitat_info = habitat_info[~habitat_info['habitat'].isna()].copy()
habitat_info = _clean_data(habitat_info)

habitat_info['lat'] = gpd.GeoDataFrame(habitat_info, geometry='geometry').centroid.y
habitat_info['lon'] = gpd.GeoDataFrame(habitat_info, geometry='geometry').centroid.x

text_cleaned = habitat_info[['occurrenceID', 'species' ,'habitat']].copy()
text_cleaned_w_geo = habitat_info[['occurrenceID', 'species' ,'habitat', 'lat', 'lon']].copy()

/tmp/ipykernel_720315/1592039262.py:2: DtypeWarning: Columns (49,56,57,59,60,61,63,64,65,66,67,68,73,74,77,78,79,83,89,91,92,95,97,98,99,100,101,102,103,105,106,107,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,135,137,138,140,141,142,143,144,145,146,147,149,150,151,152,153,154,155,160,162,163,164,167,168,171,173,174,177,178,182,183,184,185,186,189,190,191,192,193,194,195,196,197,198,199,200,204,208,209,210,211,220,221) have mixed types. Specify dtype option on import or set low_memory=False.
  habitat_info = pd.read_csv(habitat_info_path, sep="\t", on_bad_lines='skip')
/scratch/cher/Sat2Habitat/data_prep/geocell/gbif_utils.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stateProvince'] = data['

In [90]:
text_cols = ['habitat', 'habitat_wiki', 'distribution and habitat_wiki', 'description_wiki', 'ecology_wiki', 'distribution_wiki', 'header_wiki']

all occurrences within remote image grouped

In [137]:
def overhead_image_to_text_df_grouped(grid_key_df, text_cleaned, species_wiki_df, text_cols):
    '''
    Add species wiki info.
    Merge dataframes on 'occurrenceID' to ensure we have all habitat text information
    Group by occurrenceID to combine all habitat text
    '''
    text_w_wiki = text_cleaned.merge(species_wiki_df, on='species', how='left')
    merged_df = pd.merge(grid_key_df, text_w_wiki, on="occurrenceID")

    grouped_data = []
    for col in text_cols:
        merged_df[col] = merged_df[col].fillna('').astype(str)
        grouped_col = merged_df.groupby("key")[col].apply(' '.join).reset_index()
        grouped_col[col] = grouped_col[col].str.strip()
        grouped_col.rename(columns={col: f"grouped_{col}"}, inplace=True)
        grouped_data.append(grouped_col)

    grouped_df = grouped_data[0]
    for df in grouped_data[1:]:
        grouped_df = pd.merge(grouped_df, df, on="key", how="left")
    
    return grouped_df

grouped_df = overhead_image_to_text_df_grouped(grid_key_df, text_cleaned, species_wiki_df, text_cols)
grouped_df.to_csv('/scratch/cher/Sat2Habitat/data/gridkey2text_grouped.csv')

In [ ]:
# example!
grouped_df['grouped_text'][908]

'Crevices in bark of red oak // On humus on rock ledge by headwaters // Damp, shaded pockets in schist bluff // Rock bottom of fast-flowing stream // By headwaters // wet soil along stream'

each occurrence seperately, but uses grid which has consistent remote images.

In [138]:
text_w_geo_wiki = text_cleaned_w_geo.merge(species_wiki_df, on='species', how='left')
merged_df_w_geo = pd.merge(grid_key_df, text_w_geo_wiki, on="occurrenceID")

merged_df_w_geo.to_csv('/scratch/cher/Sat2Habitat/data/gridkey2text.csv')

BandAid: Make Train and Validation dataset

In [4]:
train = pd.read_csv('/data/cher/Sat2Habitat/data/crisp/train_10.csv')
val = pd.read_csv('/data/cher/Sat2Habitat/data/crisp/val_10.csv')

In [13]:
import os
train_ims = [i.replace('.jpg', '') for i in os.listdir('/data/cher/Sat2Habitat/data/bing_train_10p/')]
val_ims = [i.replace('.jpg', '') for i in os.listdir('/data/cher/Sat2Habitat/data/bing_train_10p/')]

In [18]:
# Append train and val dataframes
combined_df = pd.concat([train, val])

# Subset the combined dataframe to only include 'key' in train_ims and val_ims
new_train_df = combined_df[combined_df['key'].astype(str).isin(train_ims)]
new_val_df = combined_df[combined_df['key'].astype(str).isin(val_ims)]

In [25]:
new_val_df.to_csv('/data/cher/Sat2Habitat/data/crisp/val_10-tst.csv')